In [12]:
from bs4 import BeautifulSoup

In [13]:
import urllib.request

In [14]:
from urllib.parse import quote

In [15]:
import pandas as pd

https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=" + str(mode) + "&target=after

In [27]:
def get_movie_reviews(mcode, page_num=10):
    movie_review_df = pd.DataFrame(columns=('Title','Score', 'author'))
    url = "https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=" + str(mcode) + "&target=after"
    idx = 0

    for _ in range(0, page_num):
        movie_page = urllib.request.urlopen(url).read()
        movie_page_soup = BeautifulSoup(movie_page,'html.parser')
        review_list = movie_page_soup.find_all('td', {'class':'title'})

        for review in review_list:
            title = review.find('a',{'class':'movie color_b'}).get_text()
            score = review.find('em').get_text()
            author = review.find('td.num > a.author').get_text()
            movie_review_df.loc[idx] = [title, score, author]
            idx += 1
            print('#', end='')

        try:
            url = "https://movie.naver.com" + movie_page_soup.find('a', {'class':'pg_next'}).get('href')
#             print(url)
        except:
            break
        
    return movie_review_df

In [28]:
movie_review_df = get_movie_reviews(18847, 10)
movie_review_df

NameError: name 'review' is not defined

현재상영작 영화 리뷰 스크래핑

In [ ]:
url = "https://movie.naver.com/movie/point/af/list.naver"
naver_movie = urllib.request.urlopen(url).read()
soup = BeautifulSoup(naver_movie, 'html.parser')
select = soup.find('select', {'id':'current_movie'})
movies = select.find_all('option')

movies_dict = {}
for movie in movies[1:]:
    movies_dict[movie.get('value')] = movie.get_text()
    
movie_review_df = pd.DataFrame(columns=('Title','Score'))
for mcode in movies_dict:
    df = get_movie_reviews(mcode, 1)
    movie_review_df = pd.concat([movie_review_df, df])
    
movie_review_df